# Internet Access in New Orleans
This webpage was created by Jacob Masur and Daniel Cohen to investigate the distribution
of internet access in New Orleans across geographic regions. We would like to use geographic
data to understand how other factors such as race and income affect who has access to the
internet.


In [1]:
print("This is going well")


This is going well
